<a href="https://colab.research.google.com/github/robicastilho/remedio_certo/blob/main/remedio_certo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
#Instale a SDK Google Gen AI
!pip -q install google-genai
!pip install -q google-adk
!pip install -q python-magic

#Imports e configuração do Colab
from google import genai
from google.genai import types
from google.colab import files
from google.colab import userdata
import os

from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings
import magic # Para detectar o MIME type (instale com: pip install python-magic)

warnings.filterwarnings("ignore")

###########################################################
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
###########################################################
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

###########################################################
# Função auxiliar similar a anterior, mas que envia também o caminho de um arquivo (Fornecida pelo Gemini)
###########################################################
def call_agent_multimodal(agent: Agent, message_text: str, image_file_path: str = None) -> str:
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1") # Adapte IDs se necessário
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)

    list_of_parts = [types.Part(text=message_text)]

    if image_file_path:
        try:
            # Detectar o MIME type
            mime_type = magic.from_file(image_file_path, mime=True)
            with open(image_file_path, "rb") as f:
                image_data = f.read()
            list_of_parts.append(types.Part(inline_data=types.Blob(mime_type=mime_type, data=image_data)))
        except Exception as e:
            print(f"Erro ao processar a imagem {image_file_path}: {e}")
            # Decide como lidar com o erro, talvez levantar uma exceção ou prosseguir sem a imagem

    content = types.Content(role="user", parts=list_of_parts)

    final_response = ""
    for event in runner.run(user_id="user1", session_id="session1", new_message=content): # Adapte IDs se necessário
        if event.is_final_response():
            for part in event.content.parts:
                if part.text is not None:
                    final_response += part.text
                    final_response += "\n"
    return final_response.strip()


from typing import List
# Supondo que 'Agent', 'Google Search', 'call_agent', 'InMemorySessionService', 'Runner', e 'types'
# já foram definidos ou importados como nos seus exemplos.
# Exemplo de importações que seriam necessárias (adapte conforme sua estrutura de projeto):
# from google.ai.generativelanguage import Part, Content, HarmCategory, HarmBlockThreshold
# from vertexai.preview.generative_models import Tool, FunctionDeclaration
# from ... import Agent, InMemorySessionService, Runner, types # (dependendo da localização)
# Google Search = Tool(function_declarations=[...]) # Supondo que Google Search é um Tool

###################################################
# --- Agente: Farmacêutico Decifrador de Receitas --- #
###################################################
def agente_farmaceutico(file_path: str, especialidade_medica: str, sintomas_paciente: str):
    """
    Cria e executa um agente farmacêutico para analisar uma imagem de receita médica.

    Args:
        file_path (str): O caminho para o arquivo de imagem da receita médica.
                         IMPORTANTE: A função call_agent precisará ser adaptada
                         para carregar e enviar este arquivo como um Part de imagem.
        especialidade_medica (str): A especialidade do médico prescritor.
        sintomas_paciente (str): Os sintomas relatados pelo paciente.

    Returns:
        str: A análise da receita feita pelo agente.
    """
    farmaceutico = Agent(
        name="agente_farmaceutico",
        model=model_name,

        instruction=f"""
          Você é um Farmacêutico altamente experiente e especializado em decifrar receitas médicas manuscritas, com profundo conhecimento em farmacologia, caligrafia médica e abreviações comuns.

          **Você já tem a informação de que a especialidade médica do prescritor é {especialidade_do_prescritor} e os sintomas do paciente {sintomas_paciente}.**

          Sua tarefa principal é analisar uma imagem de uma receita médica manuscrita e extrair as informações cruciais sobre os medicamentos prescritos.

          **Para cada item identificado na receita, forneça as seguintes informações em um formato claro e objetivo:**

          * **Medicamento:** Nome do medicamento (se houver incerteza, inclua possíveis alternativas e uma observação).
          * **Posologia:** Dose, via de administração e frequência (ex: "500mg, via oral, a cada 8 horas").
          * **Duração do Tratamento:** Período pelo qual o medicamento deve ser administrado (ex: "por 7 dias").
          * **Grau de Confiança:** (ALTO, MÉDIO, BAIXO) na sua identificação, para cada campo (Medicamento, Posologia, Duração).
          * **Observações/Incertezas:** Detalhes sobre qualquer dificuldade na leitura, caracteres ilegíveis ou suposições feitas.

          ---

          **Passos para a identificação (em ordem de prioridade):**

          1.  **Decifração Direta:** Comece lendo e transcrevendo o que o médico escreveu com a maior precisão possível.
          2.  **Análise de Contexto e Caligrafia:** Utilize seu conhecimento em farmacologia, abreviações comuns em receitas e, crucialmente, a **informação da especialidade médica e sintomas do paciente** para inferir palavras ou partes ilegíveis.
          3.  **Busca por Similares (Grafia):** Se a palavra for incerta ou o medicamento encontrado não seja para tratamento dos sintomas informados, pesquise por nomes de medicamentos ou termos de posologia com grafia similar ou fonética próxima que se encaixem no contexto.
          4.  **Identificação de Ilegíveis:** Se, após os passos anteriores, uma parte da receita permanecer impossível de decifrar, classifique-a como "Não identificado" ou "Ilegível" e atribua um grau de confiança "BAIXO". **Nunca invente informações ou nomes de medicamentos.**
          5.  **Validação e Refinamento Contextual:**
              * Com base no nome do medicamento que você identificou (mesmo que com confiança baixa), **pesquise no (google_search) por medicamentos que tenham nomes iguais ou muito similares**.
              * Simultaneamente, avalie se os medicamentos encontrados nessa pesquisa e o medicamento originalmente identificado **possuem ação terapêutica compatível com os sintomas informados pelo paciente e são comumente prescritos por médicos com a mesma especialidade** para essa condição.
              * Se esta análise contextual sugerir fortemente um medicamento diferente do inicialmente decifrado (mas graficamente similar e contextualmente mais plausível), **corrija a informação do medicamento** e ajuste o Grau de Confiança para cima, justificando a alteração em "Observações/Incertezas". Caso a compatibilidade seja baixa, mas a legibilidade da receita persista, mantenha a incerteza e o Grau de Confiança original ou até o diminua.
          ---
        """,
        description="Agente farmacêutico que analisa receitas médicas manuscritas e usa o Google Search para validação.",
        tools=[google_search] # Garanta que 'Google Search' esteja definido como uma Tool
    )

    # Esta entrada de texto contextualiza a imagem que DEVE ser enviada à parte.
    entrada_do_agente_farmaceutico = (
        f"Especialidade médica do prescritor: {especialidade_medica}\n"
        f"Sintomas do paciente: {sintomas_paciente}\n\n"
        "Por favor, analise a IMAGEM da receita médica que acompanha esta solicitação e siga as instruções detalhadas."
    )

    # IMPORTANTE: A função call_agent precisará ser modificada para aceitar
    # o file_path, carregar a imagem e passá-la como um 'Part' de imagem
    # junto com o 'Part' de texto.
    # Exemplo conceitual de como seria a chamada (a definir em call_agent):
    # analise_da_receita = call_agent(farmaceutico, entrada_do_agente_farmaceutico, image_file_path=file_path)

    # Por enquanto, chamando como nos exemplos anteriores, mas ciente da limitação:
    # Esta chamada NÃO envia a imagem com a função call_agent atual.
    analise_da_receita = call_agent_multimodal(farmaceutico, entrada_do_agente_farmaceutico, image_file_path=file_path)

    return analise_da_receita

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
model_name = "gemini-2.0-flash"
#model_name = "gemini-2.5-flash-preview-04-17"

#Widget de upload
print("👆 Escolha o arquivo de imagem (receita médica ou não):")
uploaded = files.upload()
image_path = next(iter(uploaded.keys()))

#cria variavel para armazenar o prompt
especialidade_do_prescritor = input('Especialidade: ')
sintomas_paciente = input('Sintomas: ')

receita_analisada = agente_farmaceutico(image_path, especialidade_do_prescritor, sintomas_paciente)
print(receita_analisada);


👆 Escolha o arquivo de imagem (receita médica ou não):


Saving receita-medica-exemplo-coceira-genital-urologista.jpg to receita-medica-exemplo-coceira-genital-urologista (7).jpg
Especialidade: urologista
Sintomas: coceira genitaria
Aqui está a análise da receita médica, considerando a especialidade do médico (urologista) e os sintomas do paciente (coceira genital):

*   **Medicamento:** Lopox [Alto]
*   **Posologia:** Uso local, 2x ao dia [Alto]
*   **Duração do Tratamento:** Não identificável [Baixo]
*   **Grau de Confiança:** ALTO para Medicamento e Posologia, BAIXO para Duração.
*   **Observações/Incertezas:** A duração do tratamento não está legível na receita.


A pesquisa por "Lopox para coceira genital" e "Lopox pomada urologia posologia" retornou resultados relevantes.

*   **Medicamento:** Loprox (Ciclopirox Olamina) [ALTO]
*   **Posologia:** Uso local, 2 vezes ao dia [ALTO]
*   **Duração do Tratamento:** Não identificável [BAIXO]
*   **Grau de Confiança:** ALTO para Medicamento e Posologia, BAIXO para Duração.
*   **Observações/In